In [229]:
import re

#Make the dictionary from dic_ru file
with open('dic_ru.txt') as f:
    dic_list = f.readlines()
dic_list = [x.strip('\n') for x in dic_list]
dic = []
for x in dic_list:
    str_list = x.split()
    if len(str_list) == 3:
        #print(str_list[0], str_list[2])        
        dic.append([str_list[0], float(str_list[2])])

#print(dic)

In [230]:
#Ordered output to the file
with open ('dic_learn_ru.txt', 'w') as fp:    
    for x in dic:
        fp.write("%s %s %s\n" % (dic.index(x), x[0], x[1]))

#Make the dictionary from sent_dic_ru file
with open('sent_dic_ru.txt') as f:
    sent = f.readlines()
sent = [x.strip('\n') for x in sent]                


sent_list = []
for x in sent:      
    #Split into words
    str_list = x.split()
    
    #List from sentenses (words in list)
    sent_list.append(str_list)

In [231]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
def norm(word):
    p = morph.parse(word)[0]
    return p.normal_form, p.tag.POS

In [232]:
#Normalization and stop-words delete from sent_list
sent_list_norm = []
for sent in sent_list:
    one_sent_list = []
    one_sent = ()
    if abs(float(sent[-1])) > 0.3:
        for x in sent[:-1]:
            x_norm = norm(x)        
            if x_norm[1] not in ['PREP', 'PRCL', 'CONJ', 'NPRO']:
                one_sent_list.append(x_norm[0])
        weight = float(sent[-1])
        if weight > 0:
            weight_norm = 1
        else:
            weight_norm = -1
        one_sent = (one_sent_list, weight_norm)
        sent_list_norm.append(one_sent)
#print(sent_list_norm)    

In [233]:
import math

n_train = math.trunc(0.8*len(sent_list_norm))
n_test = len(sent_list_norm) - n_train
print(n_train, n_test)

60404 15102


In [234]:
import collections

#fp1 = open ('vect_sent.txt', 'w')
vect_sent = []
vect_label = []
for x in sent_list_norm:
    cnt = collections.Counter(x[0])
    vect = []
    for y in dic:       
        if y[0] in x[0]:            
            vect.append(cnt[y[0]])            
        else:
            vect.append(0)                
    vect_sent.append(vect)
    vect_label.append(x[1])
#fp1.write("%s %s\n" % (vect_sent, vect_weight))
#print(vect_sent)

In [235]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

vect_sent = np.array(vect_sent)
#print(vect_sent)
vect_label = np.array(vect_label)
print(vect_label)
print(vect_label[:-n_test])

[ 1  1  1 ...,  1  1 -1]
[ 1  1  1 ..., -1  1  1]


In [236]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC(kernel='linear')
clf.fit(vect_sent[:-n_test], vect_label[:-n_test])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [237]:
vect_label_test = []
for x in vect_sent[n_train:]:
    vect_label_test.extend(clf.predict(x).tolist())    
#print(vect_label_test)

In [238]:
vect_label_true = []
for x in vect_label[n_train:]:
    vect_label_true.append(x.tolist())    
#print(vect_label_true)

In [240]:
yes = 0
no = 0
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(n_test):    
    if vect_label_test[i] ==  vect_label_true[i]:        
        yes += 1
        if vect_label_test[i] == 1:
            TP += 1
        else:
            TN += 1
    else:
        no += 1
        if (vect_label_test[i] == 1) and (vect_label_true[i] == -1):
            FP += 1        
        else:
            FN += 1            
PR = yes/n_test
RC = yes/yes
F1 = (2*PR*RC)/(PR+RC)
ACC = (TP + TN)/(TP + FP + TN + FN)
print('PR =', PR*100, '%')
print('RC =', RC*100, '%')
print('F1 =', F1)
print('TP =', TP, 'TN =', TN, 'FP =', FP, 'FN =', FN)
print('ACC =', ACC)

PR = 99.98013508144616 %
RC = 100.0 %
F1 = 0.9999006655408761
TP = 10021 TN = 5078 FP = 1 FN = 2
ACC = 0.9998013508144616
